In [11]:
import tensorflow as tf
from tensorflow import keras
import os
import tempfile
from datetime import datetime
import pickle
import time

import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import random as python_random


import pennylane as qml
from pennylane import numpy as np

from qiskit import QuantumCircuit

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_aer.noise import NoiseModel
from qiskit.visualization import plot_histogram

In [16]:
@qml.qnode(dev)
def qnode(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.BasicEntanglerLayers(weights, wires=range(n_qubits))
    
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]

In [ ]:
@qml.qnode(dev)
def qnode(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]

qlayer = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=n_qubits)

In [17]:
QiskitRuntimeService.save_account(
    channel="ibm_cloud",
    token="Sr8ouZGkvGbD01_1N8IUvoqY4BBTbll8DMPk63h8E_qO",
    instance='ibm_quantum',
    overwrite=True
)

service = QiskitRuntimeService()
backend = service.backend("ibm_sherbrooke")  

noise_model = NoiseModel.from_backend(backend)
coupling_map = backend.configuration().coupling_map
basis_gates = noise_model.basis_gates

dev = qml.device('qiskit.aer', wires=n_qubits, backend='aer_simulator', 
                 coupling_map=coupling_map, basis_gates=basis_gates, noise_model=noise_model)


qlayer = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=n_qubits)


/home/fmazdar/.local/lib/python3.11/site-packages/pennylane/qnn/keras.py:317: PennyLaneDeprecationWarning: The 'KerasLayer' class is deprecated and will be removed in v0.42. 
  warnings.warn(


In [24]:
num_data = 1000  #1803333
EPOCHS = 100
BATCH_SIZE = 50 #2048
SEED=123

n_qubits = 2
n_layers = 1 #6
weight_shapes = {"weights": (n_layers, n_qubits)}


METRICS = [
      #keras.metrics.TruePositives(name='tp'),
      #keras.metrics.FalsePositives(name='fp'),
      #keras.metrics.TrueNegatives(name='tn'),
      #keras.metrics.FalseNegatives(name='fn'), 
      #keras.metrics.BinaryAccuracy(name='accuracy'),
      #keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      #keras.metrics.AUC(name='auc'),
]


test_features = np.array( pd.read_csv("benchmark-datasets/n" + str(num_data) + "_test_features.csv") )
test_labels = np.array( pd.read_csv("benchmark-datasets/n" + str(num_data) + "_test_labels.csv") )

train_features = np.array( pd.read_csv("benchmark-datasets/n" + str(num_data) + "_train_features.csv") )
train_labels = np.array( pd.read_csv("benchmark-datasets/n" + str(num_data) + "_train_labels.csv") )

val_features = np.array( pd.read_csv("benchmark-datasets/n" + str(num_data) + "_val_features.csv") )
val_labels = np.array( pd.read_csv("benchmark-datasets/n" + str(num_data) + "_val_labels.csv") )


os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(SEED)
python_random.seed(SEED)
tf.random.set_seed(SEED)
    
model = keras.Sequential([
    keras.layers.Dense(n_qubits, activation='relu', input_shape=(train_features.shape[-1],)),
    keras.layers.Dropout(0.5),
    qlayer,
    keras.layers.Dense(1, activation='sigmoid',bias_initializer=None)
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=METRICS)

#early_stopping= tf.keras.callbacks.EarlyStopping(monitor='val_auc', verbose=1,patience=100,mode='max',restore_best_weights=True)
    
history = model.fit(
        train_features,
        train_labels,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        #callbacks=[early_stopping],
        validation_data=(val_features, val_labels))


train_predictions = model.predict(train_features, batch_size=BATCH_SIZE)
test_predictions = model.predict(test_features, batch_size=BATCH_SIZE)


baseline_results = model.evaluate(test_features, test_labels, batch_size=BATCH_SIZE, verbose=0)

Epoch 1/100


/home/fmazdar/.local/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: Exception encountered when calling KerasLayer.call().

[1mCannot convert a partially known TensorShape <unknown> to a Tensor.[0m

Arguments received by KerasLayer.call():
  • inputs=tf.Tensor(shape=(None, 2), dtype=float32)